In [3]:
import pandas as pd
import yfinance as yf
import yahoo_fin.stock_info as si
from yahoo_fin.stock_info import get_data

1. Exploration

We can start by checking how much tickers there are in the NASDAQ stocks market:

In [2]:
nas_list=si.tickers_nasdaq()
print("Tickers in Nasdaq:", len(nas_list))
nas_list[0:15]

Tickers in Nasdaq: 4798


['AACG',
 'AADI',
 'AADR',
 'AAL',
 'AAME',
 'AAOI',
 'AAON',
 'AAPB',
 'AAPD',
 'AAPL',
 'AAPU',
 'AAXJ',
 'ABAT',
 'ABCL',
 'ABCS']

We can take as an example APPLE:

In [7]:
nas_aapl= get_data("aapl", start_date="12/04/2009", end_date="12/04/2019", index_as_date = True, interval="1wk")
nas_aapl

,open,high,low,close,adjclose,volume,ticker
2009-11-30,7.050714,7.138571,6.795714,6.904286,5.824092,1275604400,AAPL
2009-12-07,6.904286,7.132143,6.738571,6.952500,5.864760,3009384000,AAPL
2009-12-14,6.977500,7.053929,6.821429,6.979643,5.887658,2265844000,AAPL
2009-12-21,7.001786,7.476786,6.988214,7.465714,6.297683,1807836800,AAPL
2009-12-28,7.561429,7.641071,7.439643,7.526071,6.348596,1854266400,AAPL
...,...,...,...,...,...,...,...
2019-11-04,64.332497,65.110001,63.842499,65.035004,62.861572,423960800,AAPL
2019-11-11,64.574997,66.445000,64.570000,66.440002,64.412422,461333600,AAPL
2019-11-18,66.449997,67.000000,65.099998,65.445000,63.447777,455825200,AAPL
2019-11-25,65.677498,67.000000,65.625000,66.812500,64.773552,301081200,AAPL


2. Pre-processing

3. Analysis and visualizations